# Amazon MTurk Result Count

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import pandas as pd
import json
import os
import sys
import glob

import collections

In [3]:
def common_answer(amt_answers_count):
    
    most_common_ans = amt_answers_count.most_common()
    
    if most_common_ans[0][1] >= 3:
        ans = most_common_ans[0][0]

    elif most_common_ans[0][1] == 2:
        if most_common_ans[1][1] == 2:
            ca = [most_common_ans[0][0], most_common_ans[1][0]]

            if "A" in ca and "B" in ca:
                ans = "both"
            elif "A" in ca and "both" in ca:
                ans = "A"
            elif "B" in ca and "both" in ca:
                ans = "B"
            elif "A" in ca and "neither" in ca:
                ans = "A"
            elif "B" in ca and "neither" in ca:
                ans = "B"
            elif "both" in ca and "neither" in ca:
                ans = "both"
        elif most_common_ans[1][1] <= 1:
            ans = most_common_ans[0][0]
            
    assert ans in ["A", "B", "both", "neither"]

    return ans

In [4]:
def print_result(proc_result_filename):

    proc_result = pd.read_csv(proc_result_filename)
    
    num_stories = len(proc_result)
    num_answers = num_stories * 5

    # 5 workers agreement
    both = 0
    neither = 0
    model_win = 0
    gt_win = 0

    # cumulative
    model_win_cumulative = 0
    gt_win_cumulative = 0
    both_cumulative = 0
    neither_cumulative = 0
    
    # questionid_list
    model_win_id_list = []
    gt_win_id_list = []
    both_id_list = []
    neither_id_list = []

    for index, row in proc_result.iterrows():
        model = row["Generated"]
        gt = row["GT"]
        amt_answers = row["amt_answers"].split()
        questionid = row["questionid"]

        amt_answers_count = collections.Counter(amt_answers)

        most_common_ans = common_answer(amt_answers_count)
                
        if most_common_ans == "both":
            both += 1
            both_id_list.append(questionid)
        elif most_common_ans == "neither":
            neither += 1         
            neither_id_list.append(questionid)
        elif most_common_ans == model:
            model_win += 1
            model_win_id_list.append(questionid)
        else: 
            gt_win += 1
            gt_win_id_list.append(questionid)

        # model wins
        if model == "A":
            model_win_cumulative += amt_answers_count["A"]            
            gt_win_cumulative += amt_answers_count["B"]
        elif model == "B":
            model_win_cumulative += amt_answers_count["B"]            
            gt_win_cumulative += amt_answers_count["A"]

        # "Both"  
        both_cumulative += amt_answers_count["both"]

        # "Neither"
        neither_cumulative += amt_answers_count["neither"]

    print("----- Most Common Answer in each story -----")
    print("Model wins: {} / {}".format(model_win, num_stories))
    print("GT    wins: {} / {}".format(gt_win, num_stories))
    print("The number of 'both' : {} / {}".format(both, num_stories))
    print("The number of 'neither' : {} / {}".format(neither, num_stories))
    
    print("----- Cumulative -----")
    print("Model  wins : {} / {}".format(model_win_cumulative, num_answers))
    print("GT     wins : {} / {}".format(gt_win_cumulative, num_answers))
    print("The number of 'both' : {} / {}".format(both_cumulative, num_answers) )
    print("The number of 'neither' : {} / {}".format(neither_cumulative, num_answers) )
    print("-----")
    
    return {"model_win_id_list": model_win_id_list, 
            "gt_win_id_list":gt_win_id_list, 
            "both_id_list":both_id_list, 
            "neither_id_list":neither_id_list}

## Count Result

In [5]:
id_lists_dict = print_result("./proc/proc_result_from0to199.csv")

----- Most Common Answer in each story -----
Model wins: 8 / 200
GT    wins: 148 / 200
The number of 'both' : 44 / 200
The number of 'neither' : 0 / 200
----- Cumulative -----
Model  wins : 77 / 1000
GT     wins : 639 / 1000
The number of 'both' : 248 / 1000
The number of 'neither' : 36 / 1000
-----


In [6]:
import json
with open("./proc/id_lists_dict.json", "w") as jw:
    json.dump(id_lists_dict, jw, indent=4)